# Clustering

## $k$-means clustering

*$k$-means* is a classic method for clustering.

$k$ is an integer number that produces a fixed number of cluster, which are associated with a center and each data point is assigned to a cluster.

*$k$-means* solves the following optimization problem:
$$
\mathrm{minimize} \sum^{n}_{i=1} \Vert( \mathbf{x}_i - \mathbf{\mu}_{z_i} \Vert^2)  \quad \mathrm{w.r.t} \quad \left(\mathbf{\mu}, z\right)
$$
where $\mu_k$ is the center of the $k^\mathrm{th}$ cluster, $z_i$ is an index of the cluster for point $\mathbf{x}_i$

# import required library

In [ ]:
import RDatasets, Clustering, Plots
import Statistics, Distances
Plots.plotly();

### load `iris` dataset from R datasets

In [ ]:
iris = RDatasets.dataset("datasets", "iris");

### select some data for clustering

In [ ]:
features = Matrix(iris[:, 1:4])'; # features to use for clustering

### run clustering algorithm

In [ ]:
result = Clustering.kmeans(features, 3); # run K-means for the 3 clusters

### check if the number of clusters are same as specified

In [ ]:
@assert Clustering.nclusters(result) == 3

### view some basic features for $k$-meanus clustering

#### center of the clusters

In [ ]:
M = result.centers

### cluster size ==> number of data points for each cluster

In [ ]:
size = Clustering.counts(result)

### get the assignments of points to clusters

In [ ]:
a = Clustering.assignments(result)

### plot with the point color mapped to the assigned cluster index

In [ ]:
Plots.scatter(iris.PetalLength, iris.PetalWidth, marker_z=result.assignments,
        color=:lightrainbow, legend=false)

### We ran single $k$-means clustering but we don't know if 3 is the best cluster for this data
- for validation of $k$-means clustering results there are several metrics
- one of them is silhouette width
- others are elbow, cross tabulation, rand index, variation of information, V-measure, mutual information
- among them silhouette is the best metric; however, these metrics are dataset dependent
### Silhouette width measures the quality of each clustering by quantifying distance of each cluster from its neighboring clusters
- The Silhouette width for $i$ data point is a cosine norm:
    $$
    s_i = \frac{b_i - a_i}{\mathrm{max}\left(a_i, b_i\right)} 
    $$ 
where
- $a_i$ is the average distance from $i$ to the other points in the same cluster $z_i$
- $b_i$ is the average distance from the i to the points in the $k$-th cluster
### to compute Silhouette width, we need distance matrix of features/data

In [ ]:
#dist_function(x)  = Distances.pairwise(Distances.Euclidean(), x, dims = 2) # defines distance function
#dist = dist_function(features)
dists = Distances.pairwise(Distances.SqEuclidean(), features)

### find silhouette width

In [ ]:
sil_width = Statistics.mean(Clustering.silhouettes(result, dists))

### We did for one cluster. Now, perform analyses for multiple $k$ values

In [ ]:
cl_num    = [2, 3, 4, 5, 6, 7]
sil_width = []
for cluster in cl_num
    results          = Clustering.kmeans(features, cluster)
    silhouette_width = Statistics.mean(Clustering.silhouettes(results, dists))
    push!(sil_width, silhouette_width)
    display(silhouette_width)
end

In [ ]:
Plots.plot(cl_num, sil_width, xlabel="No. of cluster", ylabel="Silhouette width", linewidth=2)

### Here, $k = 2$ has highest Silhouette value. However, three is the closest to two. From my experience, $k=2$ provides highest silhouette width than larger $k$ values. Potential cause is two cluster can easilty demarcate the boundaries in a dataset. However, it does not mean that they represent the data accurately. So, it is better to look for $k$ value greater than `two`. Here, $k=3$ does it and also the actual data also has `three` distinct classification.

## DBSCAN (density-based spatial clustering of applications with noise)

DBSCAN extracts clusters that are subsets of the given set of points and satisfy the following properties:
- All points within the cluster are mutually density-connected, meaning that for any two distinct points $p$ and $q$ in a cluster, there exists a point $o$ sucht that both $p$ and $q$ are density reachable from $o$.
- If a point is density-connected to any point of a cluster, it is also part of that cluster.
- clusters with less than 20 points will be discarded:

In [ ]:
points = randn(3, 1000)
#clusters = Clustering.dbscan(features, 0.05, min_neighbors = 3, min_cluster_size = 20)
clusters = Clustering.dbscan(points, 0.05, min_neighbors = 4)